In [25]:
import torch
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
import torchvision.transforms as T
import torch.nn as nn
from torchvision.utils import make_grid
from torchvision.utils import save_image
from IPython.display import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from skimage import io, transform
import random
import scipy
import cv2

!pip install torchinfo
from torchinfo import summary

%matplotlib inline

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_alexnet = models.alexnet(pretrained=True)

In [3]:
summary(model_alexnet, (1, 3, 224, 224))

Layer (type:depth-idx)                   Output Shape              Param #
AlexNet                                  --                        --
├─Sequential: 1-1                        [1, 256, 6, 6]            --
│    └─Conv2d: 2-1                       [1, 64, 55, 55]           23,296
│    └─ReLU: 2-2                         [1, 64, 55, 55]           --
│    └─MaxPool2d: 2-3                    [1, 64, 27, 27]           --
│    └─Conv2d: 2-4                       [1, 192, 27, 27]          307,392
│    └─ReLU: 2-5                         [1, 192, 27, 27]          --
│    └─MaxPool2d: 2-6                    [1, 192, 13, 13]          --
│    └─Conv2d: 2-7                       [1, 384, 13, 13]          663,936
│    └─ReLU: 2-8                         [1, 384, 13, 13]          --
│    └─Conv2d: 2-9                       [1, 256, 13, 13]          884,992
│    └─ReLU: 2-10                        [1, 256, 13, 13]          --
│    └─Conv2d: 2-11                      [1, 256, 13, 13]         

In [5]:
# val_data_dir = r"C:\Users\ameyv\BTP\val"
# transform = transforms.Compose([transforms.CenterCrop(224), transforms.ToTensor()])
# val_data = datasets.ImageFolder(val_data_dir, transform=transform)
# dataloader = torch.utils.data.DataLoader(val_data, batch_size=100, shuffle=True)

In [53]:
class ImageDataset(Dataset):
    def __init__(self,df,img_folder,transform):
        self.df=df
        self.transform=transform
        self.img_folder=img_folder
        self.image_names=self.df[:]['name']
        self.labels=self.df[:]['label']

    def __len__(self):
        return len(self.image_names)

    def __getitem__(self,index):
        number_str = str(index)
        zero_filled_number = str(number_str.zfill(5))
        file_base = '\ILSVRC_2010_val_000'
        image=cv2.imread(self.img_folder + file_base + zero_filled_number +'.JPEG')
        image=self.transform(image)
        targets=self.labels[index]
        sample = {'image': image,'labels':targets}
        return sample

In [57]:
test_set = r"C:\Users\ameyv\BTP\labels.csv"
img_folder = r"C:\Users\ameyv\BTP\val"
df = pd.read_csv(test_set)

test_transform = transforms.Compose([transforms.ToPILImage(),transforms.CenterCrop(224), transforms.ToTensor()])

test_dataset=ImageDataset(df,img_folder,test_transform)
test_dataloader=DataLoader(test_dataset, batch_size=100, shuffle=False)
print(test_dataloader)

In [56]:
# index = 46537
# number_str = str(index)
# zero_filled_number = str(number_str.zfill(5))
# file_base = "\ILSVRC_2010_val_000"
# inp_image=cv2.imread(img_folder + zero_filled_number +".JPEG")

def imshow(inp_image, title=None):
    """imshow for Tensor."""
    inp_image = inp_image.numpy().transpose((1, 2, 0))
    inp_image = np.clip(inp, 0, 1)
    plt.imshow(inp)